# 7

In [203]:
with open("input.txt") as f:
    lines = f.readlines()

First I can go through the input and make a nested dictionary to represent the directory. 
- cd / i go back to the top level
- cd x i move in one level
- cd .. i move back one level

Here I have classes to define a Directory and a File:

In [204]:
class Directory:
    def __init__(self, name):
        self.name = name
        self.size = 0
        self.contents = {}

    def add(self, file_item):
        if file_item.name not in self.contents:
            self.contents[file_item.name] = file_item

    def get(self, name): # get a directory with a given name housed within this directory
        return self.contents[name]

    # recursve helper method to find the total size
    def __total_size_helper(self, curr):
        if isinstance(curr, File):
            return curr.size
        s = 0
        for item in curr.contents.values():
            s += self.__total_size_helper(item)
        return s
        
    # recursively find the total size of this directory
    def total_size(self):
        return self.__total_size_helper(self)

class File:
    def __init__(self, name, size):
        self.name = name
        self.size = size


Class to represent the current working directory. Backed by a stack:

In [205]:
class CurrentWorkingDirectory:
    def __init__(self):
        self.main = Directory("/")
        self.arr = [self.main]
    
    def cwd(self):
        return self.arr[-1]

    def cd(self, name):
        if name == "/":
            self.arr = [self.main] # reset stack to start
        elif name == "..":
            self.arr.pop(-1)
        else:
            self.arr.append(self.cwd().get(name))

    def get_main(self): #gets the main directory
        return self.main

    def mkdir(self, name): # create a directory in the cwd
        self.cwd().add(Directory(name))
    
    def mkfile(self, name, size): # create a file in the cwd
        self.cwd().add(File(name, size))

Function to parse the input data into a Directory object

In [206]:
def parse():

    master = CurrentWorkingDirectory()

    i = 1
    while i < len(lines):
        line = lines[i].strip().split()
        instruction = line[1] # line we read will always be an instruction because we skip non-instructions
        # print(line)
        if instruction == "cd":
            master.cd(line[2])
            i += 1
        elif instruction == "ls":
            j = i + 1
            while j < len(lines):
                curr = lines[j].strip().split()
                if curr[0] == "$":
                    break
                if curr[0] == "dir":
                    master.mkdir(curr[1])
                else:
                    master.mkfile(curr[1], int(curr[0]))
                j += 1
            i = j
    return master.get_main()

Part 1

We recursively find the sum of everything that has a sum less than 100000

In [207]:
def part1():
    sum_of_sizes = [0]
    master = parse()
    helper(master, sum_of_sizes)
    return sum_of_sizes[0]

def helper(curr, s):
    if isinstance(curr, File):
        return
    size = curr.total_size()
    if size <= 100000:
        s[0] += size
    for item in curr.contents.values():
        helper(item, s)

part1()

1297159

Part 2

We need unused space of at least 30000000. Our system has total storage size of 70000000. We currently have the following empty space:

In [208]:
size_used = parse().total_size()
size_used

43636666

In [209]:
free_space_remaining = 70000000 - size_used
free_space_remaining

26363334

This means we need to free up the following amount of space to get the minimum 30000000:

In [210]:
30000000 - free_space_remaining

3636666

In [215]:
def part2():
    min_space_to_free = 30000000 - (70000000 - parse().total_size())
    master = parse()
    solution = [30000000]
    helper(master, solution, min_space_to_free)
    return solution[0]
    
def helper(curr, sol, min_space):
    if isinstance(curr, File):
        return
    size = curr.total_size()
    if size < min_space:
        return
    if size >= min_space and size < sol[0]:
        sol[0] = size
    for item in curr.contents.values():
        helper(item, sol, min_space)

part2()

3866390